In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [0]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import minmax_scale
import gc

import lightgbm as lgb

In [0]:
%cd My\ Drive/gci_competition

/gdrive/My Drive/gci_competition


In [0]:
X_train = pd.read_csv('X_train_without_numGroups1_31_100.csv')
Y_train = pd.read_csv('Y_train_without_numGroups1.csv')
X_test = pd.read_csv('X_test_without_numGroups1_31_100.csv')

In [0]:
print(X_train.shape, X_test.shape, Y_train.shape)

(356007, 100) (89019, 100) (356007, 1)


In [0]:
grid_param={'learning_rate': [0.1],
        'objective':['regression'],
        'metric':['mae'],
        'num_leaves': [16383],
        'verbose': [1],
        'random_state':[42],
        'bagging_fraction': [0.03],
        'feature_fraction': [0.85]
       }

In [0]:
reg = lgb.LGBMRegressor()

reg_gs_cv = GridSearchCV(
            reg, # 識別器
            grid_param, # 最適化したいパラメータセット
            cv = KFold(n_splits=3, shuffle=True), # 交差検定の回数
            scoring = 'neg_mean_absolute_error',
            )

reg_gs_cv.fit(
            X_train,
            Y_train,
            )

GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=True),
             error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=...
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='warn', n_jobs=None,
             param_grid={'bagging_fraction': [0.03], 'feature_fraction': [0.85],
                         'learning_rate': [0.1], 'metric': ['mae'],
                         'num_leaves': [16383], 'objective': 

In [0]:
best_param = reg_gs_cv.best_params_
best_param =pd.DataFrame(list(best_param.items()))

In [0]:
best_score = pd.DataFrame(["best_score", reg_gs_cv.best_score_])
best_param = pd.concat([best_param, best_score.T])

import datetime
dt_now = datetime.datetime.now()
dt = dt_now.strftime('%m%d%H%M')

best_param.to_csv(dt+"best_param_without_numGroups1_31_100.csv", index=False)

In [0]:
best_param

,0,1
0,bagging_fraction,0.03
1,feature_fraction,0.85
2,learning_rate,0.1
3,metric,mae
4,num_leaves,16383
5,objective,regression
6,random_state,42
7,verbose,1
0,best_score,-0.0507258


In [0]:
best_estimator = reg_gs_cv.best_estimator_